In [2]:
import torch #for creating tensor objects, backprop etc
import torch.nn.functional as F #for one_hot

In [3]:
word = [
    "aarav", "vivaan", "aditya", "vihaan", "ishaan", "arjun", "sai", "ayaan", "krishna", "rohan",
    "reyansh", "shiv", "mohit", "karthik", "lakshman", "siddharth", "manish", "vishal", "vivek", "rahul",
    "manav", "nikhil", "kunal", "anish", "yash", "abhinav", "pranav", "amit", "gaurav", "ravi",
    "raj", "surya", "harsh", "tushar", "akash", "parth", "raghav", "ramesh", "ankit", "suresh",
    "pritam", "prem", "himesh", "ashwin", "dhruv", "sandeep", "vikas", "ajay", "anil", "madhav",
    "deepak", "bhuvan", "ashok", "ram", "shivansh", "nitin", "saurabh", "udit", "shaurya", "manoj",
    "chirag", "kiran", "amitabh", "nashit", "siddhi", "ritika", "ananya", "priya", "aishwarya", "sneha",
    "radhika", "meera", "swati", "pooja", "shruti", "simran", "nisha", "sanya", "kavya", "madhuri",
    "neha", "jaya", "mitali", "sonali", "laxmi", "vidya", "komal", "shalini", "tanu", "shreya",
    "nupur", "isha", "rupa", "divya", "ritu", "vandana", "pragya", "suman", "deepika", "manju",
    "shweta", "vaishnavi", "parul", "gayatri", "aarti", "tanvi", "chhavi", "anju", "tanisha", "sakshi",
    "simran", "sonal", "ravina", "meenal", "aarti", "shivani", "ankita", "kiran", "yamini", "sonalika"
]

#A list of strings; chatgpt generated these kerala origin names to train the model

In [4]:
import string
chars=list(string.ascii_lowercase)
stoi={s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos={i+1:s for i,s in enumerate(chars)}
itos[0]="."

#since indexing is based on integers from 0 to 1,2,3.... we map each alphabet to an integer in that order itself

In [5]:
x=[]
xi=[]
y=[]
yi=[]
for w in word:
  w='.'+w+'.'
  for i in zip(w,w[1:]):
    x.append(i[0])
    xi.append(stoi[i[0]])
    y.append(i[1])
    yi.append(stoi[i[1]])

#using zip we create bigraphs and thus generating i/p and o/p labels

In [6]:
xi=torch.tensor(xi)
yi=torch.tensor(yi)
#creating tensor objects

In [7]:
xenc = F.one_hot(xi, num_classes=27).float() #for more efficient working of model, we use one_hat for classes

In [8]:
W = torch.randn((27, 27),requires_grad=True) #since its a single layer nn with 27 neruons we need a 27x27 weight matrix

In [9]:
#forward pass matrix multiplication (we use @ for product)
logits=xenc@W
counts=logits.exp()
probs=counts/counts.sum(dim=1,keepdim=True)
probs.shape

#here we consider cross entropy loss, basically sum of negative logs of each predicted probability values corresponding to actual labels (we already have them for training set)
nlls=torch.zeros(len(yi))
j=0
for i in yi:
  loss=torch.log(probs[j,i])
  nlls[j]=-loss
  j+=1

print('=========')
print('average negative log likelihood, i.e. loss =', nlls.mean().item())
LOSS=nlls.mean()

average negative log likelihood, i.e. loss = 3.606201171875


In [10]:
W.grad=None
LOSS.backward()
#backprop to calculate gradients of each weights like how they affect the final loss

In [11]:
#conducting various epochs and using manual gradient descent to converge to min loss and get corresponding wieghts
alpha=1
for i in range(10):
  W.data+=-alpha*W.grad

  #forward pass
  logits=xenc@W
  counts=logits.exp()
  probs=counts/counts.sum(dim=1,keepdim=True)
  probs.shape

  nlls=torch.zeros(len(yi))
  j=0
  for i in yi:
    loss=torch.log(probs[j,i])
    nlls[j]=-loss
    j+=1

  print('=========')
  print('average negative log likelihood, i.e. loss =', nlls.mean().item()+0.01*(W**2).mean())
  LOSS=nlls.mean()+0.01*(W**2).mean()

average negative log likelihood, i.e. loss = tensor(3.6042, grad_fn=<AddBackward0>)
average negative log likelihood, i.e. loss = tensor(3.5912, grad_fn=<AddBackward0>)
average negative log likelihood, i.e. loss = tensor(3.5783, grad_fn=<AddBackward0>)
average negative log likelihood, i.e. loss = tensor(3.5655, grad_fn=<AddBackward0>)
average negative log likelihood, i.e. loss = tensor(3.5529, grad_fn=<AddBackward0>)
average negative log likelihood, i.e. loss = tensor(3.5404, grad_fn=<AddBackward0>)
average negative log likelihood, i.e. loss = tensor(3.5280, grad_fn=<AddBackward0>)
average negative log likelihood, i.e. loss = tensor(3.5158, grad_fn=<AddBackward0>)
average negative log likelihood, i.e. loss = tensor(3.5036, grad_fn=<AddBackward0>)
average negative log likelihood, i.e. loss = tensor(3.4917, grad_fn=<AddBackward0>)


In [12]:
g = torch.Generator().manual_seed(2147483647)
for i in range(100):
  out =[]
  ix=0
  while True:
    inputencode=F.one_hot(torch.tensor([ix]),num_classes=27).float()
    logits=inputencode@W
    counts=logits.exp()
    p=counts/counts.sum(1,keepdims=True)

    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(out)

  #testing yayyy !!


['j', 'u', 'p', 'i', 'd', 'w', 'd', 'i', 'l', 'k', 'p', 'q', 'a', 'h', 'h', 'p', 'f', 'o', 'f', 'r', 'o', 'w', 'e', 'i', 'n', 'w', 'a', '.']
['s', 'h', 'i', 'n', 'w', 't', 'o', 'h', 'e', 'k', 'g', 'i', 'f', 'r', 'h', 'z', 'y', '.']
['d', 'a', 'n', 'n', 'p', 'a', 'u', 'r', 'x', 'm', 'i', 'l', 'e', 'v', 'h', 'a', 'c', 'f', 'd', 'b', 'y', 'n', 'i', 'n', 'r', 'w', 'i', 'f', 'o', 'r', 'c', 's', 'n', 'j', 'y', 'i', 'e', 'v', 'y', '.']
['a', 'r', 't', 'z', 'y', 'f', 'f', 'r', 'm', 'u', 'p', 'c', 'h', 'a', 'f', 'o', 'e', 't', 'u', 'p', 'f', 'r', 'p', 'f', 'f', 'o', 's', 'z', 'w', 'a', 'h', 'e', 'j', 'a', 'n', 'q', 'u', 'r', 'o', 'r', 'o', 'a', 'y', 'x', 'e', 'p', 'o', 'c', 'f', 'z', 'y', '.']
['s', 'a', 'b', 'w', 'i', 'w', 'e', 'j', 'n', 'm', 'y', 'i', 'f', 'm', 'w', 'y', 'n', 'f', 'r', 'p', 'a', 'n', 'c', 'p', 's', 'n', 'h', 'm', 'v', 'f', 'i', 'h', 'o', 'u', 's', 'z', 'x', 'h', 'f', 'd', 'g', 'o', 'r', 'f', 'z', 'x', 'n', 'y', 'n', 'c', 'i', 'l', 'm', 'l', 'e', 'j', 'n', 'h', 'u', 'p', 'c', 